In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools

In [33]:
url = '''https://www.gutenberg.org/ebooks/search/?sort_order=downloads&start_index=1'''

respond = requests.get(url)
book_soup = BeautifulSoup(respond.text, 'html.parser')
# Filter the a-tags to get just the titles
book_tags = book_soup.find_all('a', attrs={'class': "link"})
book_tags = [tag.attrs['href'] for tag in book_tags 
              if tag.attrs['href'].startswith('/ebook') & (tag.attrs['href'][-1].isdigit()==True)]

# Remove duplicate links
book_tags = list(dict.fromkeys(book_tags))
  #book_tags += book_tags

In [34]:
len(book_tags)

50

In [36]:

print("In total we have " + str(len(book_tags)) + " book titles") 
print(book_tags)

In total we have 50 book titles
['/ebooks/10007', '/ebooks/158', '/ebooks/4517', '/ebooks/3825', '/ebooks/521', '/ebooks/1998', '/ebooks/34901', '/ebooks/28054', '/ebooks/1934', '/ebooks/766', '/ebooks/113', '/ebooks/2500', '/ebooks/5827', '/ebooks/140', '/ebooks/3296', '/ebooks/4363', '/ebooks/8800', '/ebooks/209', '/ebooks/42324', '/ebooks/779', '/ebooks/1251', '/ebooks/3090', '/ebooks/61', '/ebooks/600', '/ebooks/10', '/ebooks/10007', '/ebooks/158', '/ebooks/4517', '/ebooks/3825', '/ebooks/521', '/ebooks/1998', '/ebooks/34901', '/ebooks/28054', '/ebooks/1934', '/ebooks/766', '/ebooks/113', '/ebooks/2500', '/ebooks/5827', '/ebooks/140', '/ebooks/3296', '/ebooks/4363', '/ebooks/8800', '/ebooks/209', '/ebooks/42324', '/ebooks/779', '/ebooks/1251', '/ebooks/3090', '/ebooks/61', '/ebooks/600', '/ebooks/10']


In [37]:
base_url = "https://www.gutenberg.org"
# Get book links
book_links = [base_url + tag  for tag in book_tags]
print(book_links[:10])

['https://www.gutenberg.org/ebooks/10007', 'https://www.gutenberg.org/ebooks/158', 'https://www.gutenberg.org/ebooks/4517', 'https://www.gutenberg.org/ebooks/3825', 'https://www.gutenberg.org/ebooks/521', 'https://www.gutenberg.org/ebooks/1998', 'https://www.gutenberg.org/ebooks/34901', 'https://www.gutenberg.org/ebooks/28054', 'https://www.gutenberg.org/ebooks/1934', 'https://www.gutenberg.org/ebooks/766']


In [38]:
# Create a helper function to get review links
def getBook(soup):
    # Get all the review tags
    book_list = soup.find_all('a', attrs={'class':'link'})
    # Get the first review tag
    format_tag = book_list[0]
    # Return the none review link
    format_link = "https://www.gutenberg.org" + format_tag['href']
    return format_link

# Get a list of soup objects. This takes a while
book_form_soups = [BeautifulSoup(requests.get(link).text, 'html.parser') for link in book_links]
# Get all 100 movie review links
book_format_list = [getBook(book_form_soup) for book_form_soup in book_form_soups]

print("There are a total of " + str(len(book_format_list)) + " individual movie reviews")
print(book_format_list[:10])

There are a total of 50 individual movie reviews
['https://www.gutenberg.org/ebooks/10007.html.images', 'https://www.gutenberg.org/ebooks/158.html.images', 'https://www.gutenberg.org/ebooks/4517.html.images', 'https://www.gutenberg.org/ebooks/3825.html.images', 'https://www.gutenberg.org/ebooks/521.html.images', 'https://www.gutenberg.org/ebooks/1998.html.images', 'https://www.gutenberg.org/ebooks/34901.html.images', 'https://www.gutenberg.org/ebooks/28054.html.images', 'https://www.gutenberg.org/ebooks/1934.html.images', 'https://www.gutenberg.org/ebooks/766.html.images']


In [39]:
# Create lists for dataframe and csv later
book_texts = []
book_titles = []
regex = re.compile('chapter.*')

# Loop through the movie reviews
for url in book_format_list:
    # Get the review page
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    # Find div tags with class text show-more__control, then get its text
    if soup.find_all('div', attrs={'class': regex}):
      chapter_tag = soup.find('div', attrs={'class': 'chapter'}).getText()
    else:
      chapter_tag = soup.find_all('p')[6].getText()
    # Add the review text in the review list
    book_texts += [chapter_tag]
    # Find the h1 tag and get the second element i.e. the title
    title_tag = soup.find('h1').getText()
    # Add the title in the title list
    book_titles += [title_tag]

In [40]:
# Construct a dataframe
df = pd.DataFrame({'book': book_titles, 'book_download_link':book_format_list,'book_chapter': book_texts})
print(df[:10])
# Put into .csv file
with open("Book_download.csv", mode='w', newline='\n') as f:
    df.to_csv(f, sep=",", line_terminator='\n', encoding='utf-8')
#df.to_csv('Book_download.csv', index=False)

                                                book  ...                                       book_chapter
0                                           Carmilla  ...  \n\nPROLOGUE\nUpon a paper attached to the Nar...
1                                               Emma  ...                                     \n\nVOLUME I\n
2                                        ETHAN FROME  ...                                                  I
3                                          PYGMALION  ...                                                   
4  The\r\nLife and Adventures\r\nof\r\nRobinson C...  ...  \n\nCHAPTER I.\r\nSTART IN LIFE\nI was born in...
5                             THUS SPAKE ZARATHUSTRA  ...                                                 \n
6               \nOn Liberty. By John Stuart Mill.\n  ...                                           [Pg vii]
7                             The Brothers Karamazov  ...                                       \n\nPART I\n
8     SONGS OF INNO

In [45]:
data=pd.read_csv('Book_download.csv')
data.head()
data.tail()

,Unnamed: 0,book,book_download_link,book_chapter
45,45,Le Morte D’Arthur,https://www.gutenberg.org/ebooks/1251.html.images,\n\nBIBLIOGRAPHICAL NOTE\nThe Morte D’Arthur w...
46,46,THE ENTIRE ORIGINAL MAUPASSANT SHORT STORIES,https://www.gutenberg.org/ebooks/3090.html.images,VOLUME I.
47,47,MANIFESTO OF THE COMMUNIST PARTY,https://www.gutenberg.org/ebooks/61.html.noimages,*** START OF THIS PROJECT GUTENBERG EBOOK THE ...
48,48,Notes from the Underground,https://www.gutenberg.org/ebooks/600.html.images,*The author of the diary and the diary itself ...
49,49,The King James Version of the Bible,https://www.gutenberg.org/ebooks/10.html.images,\n\nThe Old Testament of the King James Versio...


In [46]:
data['book_chapter']=data['book_chapter'].str.replace('\\n', ' ')
data['book_chapter']=data['book_chapter'].str.replace('\\r', ' ')
data['book_chapter']=data['book_chapter'].str.lower()
data['book']=data['book'].str.replace('\\n', ' ')
data['book']=data['book'].str.replace('\\r', ' ')
data.tail()

#data[:10]

,Unnamed: 0,book,book_download_link,book_chapter
45,45,Le Morte D’Arthur,https://www.gutenberg.org/ebooks/1251.html.images,bibliographical note the morte d’arthur was ...
46,46,THE ENTIRE ORIGINAL MAUPASSANT SHORT STORIES,https://www.gutenberg.org/ebooks/3090.html.images,volume i.
47,47,MANIFESTO OF THE COMMUNIST PARTY,https://www.gutenberg.org/ebooks/61.html.noimages,*** start of this project gutenberg ebook the ...
48,48,Notes from the Underground,https://www.gutenberg.org/ebooks/600.html.images,*the author of the diary and the diary itself ...
49,49,The King James Version of the Bible,https://www.gutenberg.org/ebooks/10.html.images,the old testament of the king james version ...
